**The goal is to create a logistic regression model to predict absenteeism based on the prepared dataset.**

In [294]:
import pandas as pd
import numpy as np

In [295]:
data_preprocessed = pd.read_csv('AbsenteeismPreprocessed.csv')

In [296]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


**Create the targets:**

In [298]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

The median of the dataset is used as a cut-off point to categorize someone into one of two groups: 'being absent too much' or 'not'. It also helps address the dataset's class imbalance.

In [300]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] >
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [301]:
#targets

In [302]:
data_preprocessed['Excessive Absenteeism'] = targets

In [303]:
#data_preprocessed.head()
data_preprocessed.describe

<bound method NDFrame.describe of      Reason_1  Reason_2  Reason_3  Reason_4  Month Value  Day of the Week  \
0           0         0         0         1            7                1   
1           0         0         0         0            7                1   
2           0         0         0         1            7                2   
3           1         0         0         0            7                3   
4           0         0         0         1            7                3   
..        ...       ...       ...       ...          ...              ...   
695         1         0         0         0            5                2   
696         1         0         0         0            5                2   
697         1         0         0         0            5                3   
698         0         0         0         1            5                3   
699         0         0         0         1            5                3   

     Transportation Expense  Distance to 

Check if dataset is balanced:

In [305]:
targets.sum() / targets.shape[0]

0.45571428571428574

In [306]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'], axis=1)

**Select the inputs for the regression:**

In [308]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

In [309]:
#unscaled_inputs.head()

**Standardize the data**

It is important to ensure that all inputs have a similar magnitude for better model performance. Some values must remain unscaled to improve the model's interpretability, such as the reasons columns, and that is why CustomScaler is needed.

In [311]:
from sklearn.preprocessing import StandardScaler

class CustomScaler:
    def __init__(self, columns_to_scale, copy=True, with_mean=True, with_std=True):
        self.columns_to_scale = columns_to_scale 
        self.scaler = StandardScaler(copy=copy, with_mean=with_mean, with_std=with_std)

    def fit(self, X):
        self.scaler.fit(X[self.columns_to_scale])
        self.mean_ = np.mean(X[self.columns_to_scale])
        self.var_ = np.var(X[self.columns_to_scale])
        return self

    def transform(self, X):
        X_scaled = X.copy()
        X_scaled[self.columns_to_scale] = self.scaler.transform(X[self.columns_to_scale])
        return X_scaled

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [312]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [313]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [314]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [315]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [316]:
absenteeism_scaler.fit(unscaled_inputs)

C:\Users\ankas\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [317]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [318]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,-0.683704,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-0.683704,-1.574681,-1.141882,2.130803,-0.806331,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.007725,-0.654143,1.426749,0.248310,-0.806331,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.668253,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,0.668253,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.007725,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,-0.007725,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,0.668253,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.668253,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,1,-0.919030,-0.589690


In [319]:
scaled_inputs.shape

(700, 14)

**Split the data into train & test subsets:**

In [321]:
from sklearn.model_selection import train_test_split

In [322]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, test_size=0.2, random_state=20)

In [323]:
print(x_train.shape, y_train.shape)

(560, 14) (560,)


In [324]:
print(x_test.shape, y_test.shape)

(140, 14) (140,)


**Import sklearn logistic regression**

In [326]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

**Training the model:**

In [328]:
reg = LogisticRegression()

In [329]:
reg.fit(x_train, y_train)

LogisticRegression()

In [330]:
reg.score(x_train, y_train)

0.775

In [331]:
reg.intercept_

array([-1.65662792])

In [332]:
reg.coef_

array([[ 2.80136327e+00,  9.33540824e-01,  3.09673857e+00,
         8.57183147e-01,  1.66403124e-01, -8.43159241e-02,
         6.13215559e-01, -7.77871894e-03, -1.65545282e-01,
        -7.68487792e-05,  2.71154773e-01, -2.06026920e-01,
         3.61897667e-01, -2.85728905e-01]])

In [333]:
feature_name = unscaled_inputs.columns.values

In [334]:
summary_table = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
#summary_table

In [335]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.656628
1,Reason_1,2.801363
2,Reason_2,0.933541
3,Reason_3,3.096739
4,Reason_4,0.857183
5,Month Value,0.166403
6,Day of the Week,-0.084316
7,Transportation Expense,0.613216
8,Distance to Work,-0.007779
9,Age,-0.165545


**Interpreting the coefficients**

In [372]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [374]:
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-1.656628,0.190781
1,Reason_1,2.801363,16.467081
2,Reason_2,0.933541,2.543499
3,Reason_3,3.096739,22.125672
4,Reason_4,0.857183,2.356513
5,Month Value,0.166403,1.181049
6,Day of the Week,-0.084316,0.919141
7,Transportation Expense,0.613216,1.846359
8,Distance to Work,-0.007779,0.992251
9,Age,-0.165545,0.847431


In [376]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature name,Coefficient,Odds_ratio
3,Reason_3,3.096739,22.125672
1,Reason_1,2.801363,16.467081
2,Reason_2,0.933541,2.543499
4,Reason_4,0.857183,2.356513
7,Transportation Expense,0.613216,1.846359
13,Children,0.361898,1.436052
11,Body Mass Index,0.271155,1.311478
5,Month Value,0.166403,1.181049
10,Daily Work Load Average,-0.000077,0.999923
8,Distance to Work,-0.007779,0.992251


**Testing the model**

In [379]:
reg.score(x_test, y_test)

0.7428571428571429

In [381]:
prediction_probability = reg.predict_proba(x_test)
prediction_probability

array([[0.73844522, 0.26155478],
       [0.60830752, 0.39169248],
       [0.40932165, 0.59067835],
       [0.80500775, 0.19499225],
       [0.07319825, 0.92680175],
       [0.31989141, 0.68010859],
       [0.31315484, 0.68684516],
       [0.1333815 , 0.8666185 ],
       [0.797184  , 0.202816  ],
       [0.75285455, 0.24714545],
       [0.48221987, 0.51778013],
       [0.19628597, 0.80371403],
       [0.07847395, 0.92152605],
       [0.7063866 , 0.2936134 ],
       [0.30657701, 0.69342299],
       [0.57044278, 0.42955722],
       [0.54149644, 0.45850356],
       [0.57202077, 0.42797923],
       [0.38158691, 0.61841309],
       [0.04856239, 0.95143761],
       [0.6977674 , 0.3022326 ],
       [0.79590534, 0.20409466],
       [0.39509601, 0.60490399],
       [0.42263896, 0.57736104],
       [0.26631148, 0.73368852],
       [0.75609167, 0.24390833],
       [0.51061432, 0.48938568],
       [0.86799512, 0.13200488],
       [0.20219374, 0.79780626],
       [0.78649073, 0.21350927],
       [0.

In [385]:
#prediction_probability.shape

**Save the model**

In [391]:
import pickle

In [393]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [395]:
with open('scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)